In [7]:
import py2neo

airport_names = ["Dublin", "Paris", "London", "Malaga"]

# visual check
print "CREATE", ",".join ( [ '({name}:Airport {{name:"{name}"}})'.format(name=a) for a in airport_names])

CREATE (Dublin:Airport {name:"Dublin"}),(Paris:Airport {name:"Paris"}),(London:Airport {name:"London"}),(Malaga:Airport {name:"Malaga"})


In [8]:
from py2neo import Graph, Relationship
g = Graph("http://localhost:7474/db/data/")
g.delete_all()

In [9]:
from py2neo import Node
try:
    #create index and ensure no duplicates
    # not actually neessary because write is a one-time operation
    g.schema.create_uniqueness_constraint("Airport", "name")
except:
    pass

nodes = [g.create(Node("Airport", name=a)) for a in airport_names]

In [10]:
# could have used the nodes above but tinkering with API
airports = list(g.find("Airport"))

In [13]:
import itertools, random

#permutations not combinations because flights can start at either Airport
potential_flights = list ( itertools.permutations ( airports, 2) )
r = random.Random(117)

for i in range(16):
    to, dest = r.choice(potential_flights)
    print to, dest
    g.create(Relationship(to, "Flies_To", dest))

(dublin:Airport {name:"Dublin"}) (malaga:Airport {name:"Malaga"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(london:Airport {name:"London"}) (dublin:Airport {name:"Dublin"})
(london:Airport {name:"London"}) (dublin:Airport {name:"Dublin"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(malaga:Airport {name:"Malaga"}) (dublin:Airport {name:"Dublin"})
(dublin:Airport {name:"Dublin"}) (paris:Airport {name:"Paris"})
(malaga:Airport {name:"Malaga"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (malaga:Airport {name:"Malaga"})
(dublin:Airport {name:"Dublin"}) (london:Airport {name:"London"})
(paris:Airport {name:"Paris"}) (london:Airport {name:"London"})
(paris:Airport {name:"

In [12]:
connecting_flights_paris_to_dublin = """
MATCH (paris:Airport) - [:Flies_To] -> (an:Airport) - [:Flies_To]-> (dublin:Airport)
WHERE paris.name='Paris' AND dublin.name = "Dublin"
RETURN paris, an, dublin
"""
results = g.run(connecting_flights_paris_to_dublin)
for r in results:
    print r['an']

(malaga:Airport {name:"Malaga"})
(london:Airport {name:"London"})
